In [ ]:
import pandas as pd
from google.colab import files
uploaded = files.upload()

# Load dataset
df = pd.read_csv("sentiment140.csv", encoding="latin-1", header=None)

# Assign column names based on dataset format
df.columns = ["target", "id", "date", "flag", "user", "text"]

# Drop unnecessary columns
df = df[["target", "text"]]

# Convert target labels (0 = Negative, 2 = Neutral, 4 = Positive)
df["target"] = df["target"].map({0: "Negative", 2: "Neutral", 4: "Positive"})

# Show sample data
df.head()

Saving sentiment140.csv to sentiment140.csv


,target,text
0,Negative,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,Negative,is upset that he can't update his Facebook by ...
2,Negative,@Kenichan I dived many times for the ball. Man...
3,Negative,my whole body feels itchy and like its on fire
4,Negative,"@nationwideclass no, it's not behaving at all...."


In [ ]:
import re
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

def clean_text(text):
    text = re.sub(r"http\S+|www\S+|@\S+|\W", " ", text)  # Remove links & special characters
    text = " ".join([word for word in text.split() if word.lower() not in stop_words])  # Remove stopwords
    return text.lower()

df["clean_text"] = df["text"].apply(clean_text)

df.head()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


,target,text,clean_text
0,Negative,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",bummer shoulda got david carr third day
1,Negative,is upset that he can't update his Facebook by ...,upset update facebook texting might cry result...
2,Negative,@Kenichan I dived many times for the ball. Man...,dived many times ball managed save 50 rest go ...
3,Negative,my whole body feels itchy and like its on fire,whole body feels itchy like fire
4,Negative,"@nationwideclass no, it's not behaving at all....",behaving mad see


In [ ]:
!pip install nltk textblob vaderSentiment
import nltk
from textblob import TextBlob
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
text = "I love this product, it's amazing!"
blob_sentiment = TextBlob(text).sentiment.polarity
vader_sentiment = SentimentIntensityAnalyzer().polarity_scores(text)["compound"]

print("TextBlob Sentiment:", blob_sentiment)
print("VADER Sentiment:", vader_sentiment)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 kB 2.2 MB/s eta 0:00:00
TextBlob Sentiment: 0.625
VADER Sentiment: 0.8516


In [ ]:

analyzer = SentimentIntensityAnalyzer()

def analyze_sentiment(text):
    sentiment = TextBlob(text).sentiment.polarity  # TextBlob score
    vader_score = analyzer.polarity_scores(text)["compound"]  # VADER score
    return (sentiment + vader_score) / 2  # Average score

df["sentiment_score"] = df["clean_text"].apply(analyze_sentiment)

df.head()

,target,text,clean_text,sentiment_score
0,Negative,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",bummer shoulda got david carr third day,-0.19090
1,Negative,is upset that he can't update his Facebook by ...,upset update facebook texting might cry result...,-0.36345
2,Negative,@Kenichan I dived many times for the ball. Man...,dived many times ball managed save 50 rest go ...,0.49695
3,Negative,my whole body feels itchy and like its on fire,whole body feels itchy like fire,-0.02500
4,Negative,"@nationwideclass no, it's not behaving at all....",behaving mad see,-0.55945


In [ ]:
def categorize_sentiment(score):
    if score > 0.1:
        return "Positive"
    elif score < -0.1:
        return "Negative"
    else:
        return "Neutral"

df["sentiment_category"] = df["sentiment_score"].apply(categorize_sentiment)

df.to_csv("sentiment140.csv", index=False)

KeyError: 'sentiment_score'